# Using the terminal for NetCDF files

Using the terminal is a very handy way to do a quick preliminary analysis of a NetCDF file. 
You can do run the following examples and exercises either using your terminal, or you can run shell commands by writing `!command` in a Jupyter Notebook cell (as is done below). If you use you terminal, remember to remove the `!` in front of each instruction.

In [1]:
# Example of running shell commands in Jupyter: Display the content of you home
!ls ~

Desktop               Music                 Zotero
Documents             OneDrive - Nexus365   slp?.png
Downloads             Pictures              ssh-debug4support.sh
Huracan               Public                ssh-debug4support.txt
Library               Softs
Movies                Teaching


In [3]:
# Task: Display the list of file(s) available in the data folder (../../data_samples/netcdf/E-OBS/)
!ls ../../data_samples/netcdf/E-OBS/

UK_monthly.nc    process.sh       rr_UK_monthly.nc
pp_UK_monthly.nc raw              tg_UK_monthly.nc


Files with extension `.nc` are "NetCDF" files. It is a standardized binary format suitable for multi-dimensional data. Binary files cannot be read as easily as the text files you have seen previously (you can try to run `head file.nc`, it will yield something, but nothing you can read with a human brain). You need specific tools to read this type of files. 

## 1. Reading the file: ncdump
`ncdump` is the most basic command to check what is in a NetCDF file. Use the `-h` or `-c` option to display the "header" of the file, which contains all the essential information. Use the `-v <var>` option to display the content of a specific variable. If you do not specify any option, the whole content of the file is displayed, which, in most cases, will be very long. If you make this mistake, you'll likely need to kill the process (Ctrl+C in the terminal, Stop button in Jupyter). 

In [5]:
# Display the header of one of the UK_monthly.nc file in the E-OBS folder
!ncdump -h ../../data_samples/netcdf/E-OBS/UK_monthly.nc

netcdf UK_monthly {
dimensions:
	latitude = 38 ;
	longitude = 52 ;
	time = UNLIMITED ; // (888 currently)
variables:
	double latitude(latitude) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "Latitude values" ;
		latitude:units = "degrees_north" ;
		latitude:axis = "Y" ;
	double longitude(longitude) ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "Longitude values" ;
		longitude:units = "degrees_east" ;
		longitude:axis = "X" ;
	float pp(time, latitude, longitude) ;
		pp:standard_name = "air_pressure_at_sea_level" ;
		pp:long_name = "sea level pressure" ;
		pp:units = "hPa" ;
		pp:_FillValue = -9999.f ;
		pp:missing_value = -9999.f ;
		pp:cell_methods = "time: mean" ;
	float rr(time, latitude, longitude) ;
		rr:standard_name = "thickness_of_rainfall_amount" ;
		rr:long_name = "rainfall" ;
		rr:units = "mm" ;
		rr:_FillValue = -9999.f ;
		rr:missing_value = -9999.f ;
		rr:cell_methods = "time: mean" ;
	float tg(time, latitude, longitude) ;
		tg:stan

**Question: Which variable does each file contain? What are the associated units? What are the dimensions of the variables?**

**Question: What is the difference between using the `-c` and `-h` commands?**

In [6]:
# Task: Display the latitude and longitude variables for one of UK_monthly.nc file in the E-OBS folder
!ncdump -v latitude ../../data_samples/netcdf/E-OBS/UK_monthly.nc
!ncdump -v longitude ../../data_samples/netcdf/E-OBS/UK_monthly.nc

netcdf UK_monthly {
dimensions:
	latitude = 38 ;
	longitude = 52 ;
	time = UNLIMITED ; // (888 currently)
variables:
	double latitude(latitude) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "Latitude values" ;
		latitude:units = "degrees_north" ;
		latitude:axis = "Y" ;
	double longitude(longitude) ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "Longitude values" ;
		longitude:units = "degrees_east" ;
		longitude:axis = "X" ;
	float pp(time, latitude, longitude) ;
		pp:standard_name = "air_pressure_at_sea_level" ;
		pp:long_name = "sea level pressure" ;
		pp:units = "hPa" ;
		pp:_FillValue = -9999.f ;
		pp:missing_value = -9999.f ;
		pp:cell_methods = "time: mean" ;
	float rr(time, latitude, longitude) ;
		rr:standard_name = "thickness_of_rainfall_amount" ;
		rr:long_name = "rainfall" ;
		rr:units = "mm" ;
		rr:_FillValue = -9999.f ;
		rr:missing_value = -9999.f ;
		rr:cell_methods = "time: mean" ;
	float tg(time, latitude, longitude) ;
		tg:stan

**Question: Over which coordinate box is the data provided?**

## 2. Visualising the file: ncview
`ncview` is a graphical tool to check the content of a NetCDF file. When you run `ncview file.nc` a new window opens with a graphical interface.

In [11]:
# Run ncview for one of the E-OBS file, and watch how the variable changes over time. 
# (You may need to click on the variable name to display it)
# NB: To get back control over your terminal or notebook, close the window. 
!ncview ../data_samples/netcdf/E-OBS/UK_monthly.nc

Ncview 2.1.8 David W. Pierce  8 March 2017
http://meteora.ucsd.edu:80/~pierce/ncview_home_page.html
Copyright (C) 1993 through 2015, David W. Pierce
Ncview comes with ABSOLUTELY NO WARRANTY; for details type `ncview -w'.
This is free software licensed under the Gnu General Public License version 3; type `ncview -c' for redistribution details.

Note: no Ncview app-defaults file found, using internal defaults
calculating min and maxes for pp...
calculating min and maxes for rr...
calculating min and maxes for tg...


**Question: What is the range of values each variable takes? Does it seem sensible?**

**Question: Over which range of dates is the data provided? What is the frequency of the data**

## 3. nco

`nco` provides a suite of commands that can be used to manipulate NetCDF files. Here we introduce the most common ones. You may find all the functions <HERE> for future reference.

* `ncks` (NetCDF kitchen sink) is used to subset from NetCDF file.
* `ncrcat` is used to concatenate files along time dimension.
* `ncra` is used to average variables over time.

Command line tools require the creation of intermediary files, that we will save in the `tmp` folder.

### Subsetting particular point or slice with `ncks`
`ncks -d dim_name,value(,value2) file_in.nc file_out.nc`
NB: If value is a integer, `ncks` will read it as an index (look for the nth value), whereas if value is a float, it will look for the closest value. Therefore, if you want to extract a specific round latitude or longitude, remember to still write `35.0` to be sure to get the value closest to 35, and not the 35th value.

In [10]:
# Extract a given time step
!ncks -d time,10 ../../data_samples/netcdf/E-OBS/UK_monthly.nc tmp/time_step_10.nc
# Extract a time slice
!ncks -d time,10,20 ../../data_samples/netcdf/E-OBS/UK_monthly.nc tmp/time_step_10_20.nc

ncks: tmp/time_step_10.nc exists---`e'xit, `o'verwrite (i.e., clobber existing file), or `a'ppend (i.e., replace duplicate variables in, and add metadata and new variables to, existing file) (e/o/a)? ^C


In [15]:
# Extract the values for Oxford
!ncks -d latitude,51.75 -d longitude,-1.26 ../data_samples/netcdf/E-OBS/UK_monthly.nc tmp/Oxford_nco.nc

ncks: ERROR file "../data_samples/netcdf/E-OBS/UK_monthly.nc" not found. It does not exist on the local filesystem, nor does it match remote filename patterns (e.g., http://foo or foo.bar.edu:file).
ncks: HINT file-not-found errors usually arise from filename typos, incorrect paths, missing files, or capricious gods. Please verify spelling and location of requested file. If the file resides on a High Performance Storage System (HPSS) accessible via the 'hsi' command, then add the --hpss option and re-try command.


In [16]:
# Extract Temperature over Ireland
!ncks -d latitude,51.0,55.5 -d longitude,-11.0,-5.0 ../data_samples/netcdf/E-OBS/UK_monthly.nc tmp/Ireland_nco.nc

ncks: ERROR file "../data_samples/netcdf/E-OBS/UK_monthly.nc" not found. It does not exist on the local filesystem, nor does it match remote filename patterns (e.g., http://foo or foo.bar.edu:file).
ncks: HINT file-not-found errors usually arise from filename typos, incorrect paths, missing files, or capricious gods. Please verify spelling and location of requested file. If the file resides on a High Performance Storage System (HPSS) accessible via the 'hsi' command, then add the --hpss option and re-try command.


In [11]:
# Task: Explore the content of the new files with ncdump and ncview
# Remark the changes in dimensions. Check that it is indeed Ireland that has been selected.
!ncdump -h tmp/Oxford_nco.nc
!ncview tmp/Oxford_nco.nc

netcdf Oxford_nco {
dimensions:
	latitude = 1 ;
	longitude = 1 ;
	time = UNLIMITED ; // (888 currently)
variables:
	double latitude(latitude) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "Latitude values" ;
		latitude:units = "degrees_north" ;
		latitude:axis = "Y" ;
	double longitude(longitude) ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "Longitude values" ;
		longitude:units = "degrees_east" ;
		longitude:axis = "X" ;
	float pp(time, latitude, longitude) ;
		pp:standard_name = "air_pressure_at_sea_level" ;
		pp:long_name = "sea level pressure" ;
		pp:units = "hPa" ;
		pp:_FillValue = -9999.f ;
		pp:missing_value = -9999.f ;
		pp:cell_methods = "time: mean" ;
	float rr(time, latitude, longitude) ;
		rr:standard_name = "thickness_of_rainfall_amount" ;
		rr:long_name = "rainfall" ;
		rr:units = "mm" ;
		rr:_FillValue = -9999.f ;
		rr:missing_value = -9999.f ;
		rr:cell_methods = "time: mean" ;
	float tg(time, latitude, longitude) ;
		tg:standa

### Statistical operations
With `ncra` you can average over the whole time of a file, by running `ncra file_in.nc file_out.nc`. It will compute the average for all variables in the file, unless you specify a specific one with `-v`: `ncra -v var file_in.nc file_out.nc`

In [22]:
# Compute the average variables over Ireland
!ncra tmp/Ireland_nco.nc tmp/Ireland_avg_nco.nc

In [17]:
# Task: Compute the average for only precipitation over Ireland
!ncra -v rr tmp/Ireland_nco.nc tmp/Ireland_rr_avg_nco.nc

In [23]:
# Task: Explore the content of your new file with ncdump and ncview
# Remark the time dimension has been reduced to 1.
!ncdump -h tmp/Ireland_avg_nco.nc
!ncview tmp/Ireland_avg_nco.nc

netcdf Ireland_avg_nco {
dimensions:
	latitude = 18 ;
	longitude = 24 ;
	time = UNLIMITED ; // (1 currently)
variables:
	double latitude(latitude) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "Latitude values" ;
		latitude:units = "degrees_north" ;
		latitude:axis = "Y" ;
	double longitude(longitude) ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "Longitude values" ;
		longitude:units = "degrees_east" ;
		longitude:axis = "X" ;
	float pp(time, latitude, longitude) ;
		pp:standard_name = "air_pressure_at_sea_level" ;
		pp:long_name = "sea level pressure" ;
		pp:units = "hPa" ;
		pp:_FillValue = -9999.f ;
		pp:missing_value = -9999.f ;
		pp:cell_methods = "time: mean" ;
	float rr(time, latitude, longitude) ;
		rr:standard_name = "thickness_of_rainfall_amount" ;
		rr:long_name = "rainfall" ;
		rr:units = "mm" ;
		rr:_FillValue = -9999.f ;
		rr:missing_value = -9999.f ;
		rr:cell_methods = "time: mean" ;
	float tg(time, latitude, longitude) ;
		tg:s

**Question: What are the rainiest and driest places in Ireland?**

## 4. cdo
`cdo` is another suite of command line tools to manipulate NetCDF files. It is more comprehensive than `nco`, but, as such, also less simple. A comprehensive documentation can be found here: http://www.idris.fr/media/ada/cdo.pdf . 
Here again, we introduce basic cdo functions. 

cdo command always start with `cdo`, then you can call one or several commands, then you name the input file(s) and finally the output file.

### Exploring the file

In [24]:
# Task: Check the dimensions of a file using cdo sinfo
!cdo sinfo ../data_samples/netcdf/E-OBS/UK_monthly.nc

cdi  warning (cdfScanVarAttr): NetCDF: Variable not found - time_bnds
   File format : NetCDF4
    -1 : Institut Source   T Steptype Levels Num    Points Num Dtype : Parameter ID
     1 : unknown  unknown  v instant       1   1      1976   1  F32  : -1            
     2 : unknown  unknown  v instant       1   1      1976   1  F32  : -2            
     3 : unknown  unknown  v instant       1   1      1976   1  F32  : -3            
   Grid coordinates :
     1 : lonlat                   : points=1976 (52x38)
                        longitude : -10.875 to 1.875 by 0.25 degrees_east
                         latitude : 49.625 to 58.875 by 0.25 degrees_north
   Vertical coordinates :
     1 : surface                  : levels=1
   Time coordinate :
                             time : 888 steps
     RefTime =  1950-01-01 00:00:00  Units = days  Calendar = standard
  YYYY-MM-DD hh:mm:ss  YYYY-MM-DD hh:mm:ss  YYYY-MM-DD hh:mm:ss  YYYY-MM-DD hh:mm:ss
  1950-01-16 00:00:00  1950-02-14 00:00:00

In [25]:
# Check the variables contained in a file using cdo showname
!cdo showname ../data_samples/netcdf/E-OBS/UK_monthly.nc

cdi  warning (cdfScanVarAttr): NetCDF: Variable not found - time_bnds
 pp rr tg
cdo    showname: Processed 3 variables [0.02s 38MB]


### Subsetting

In [28]:
# Select values for Oxford
!cdo -remapnn,lon=51.75/lat=1.26 ../data_samples/netcdf/E-OBS/UK_monthly.nc tmp/Oxford_cdo.nc

cdi  warning (cdfScanVarAttr): NetCDF: Variable not found - time_bnds
cdo    remapnn: Nearest neighbor weights from lonlat (52x38) to lonlat (1x1) grid, with source mask (624)
cdo    remapnn:   10cdo    remapnn: Nearest neighbor weights from lonlat (52x38) to lonlat (1x1) grid, with source mask (686)
cdo    remapnn:   10cdo    remapnn: Nearest neighbor weights from lonlat (52x38) to lonlat (1x1) grid, with source mask (685)
cdo    remapnn:   10cdo    remapnn: Nearest neighbor weights from lonlat (52x38) to lonlat (1x1) grid, with source mask (647)
cdo    remapnn:   10cdo    remapnn: Nearest neighbor weights from lonlat (52x38) to lonlat (1x1) grid, with source mask (680)
cdo    remapnn:   10cdo    remapnn: Nearest neighbor weights from lonlat (52x38) to lonlat (1x1) grid, with source mask (667)
cdo    remapnn:   10cdo    remapnn: Processed 5264064 values from 3 variables over 888 timesteps [0.18s 64MB]


In [29]:
# Select the same box as before with cdo sellonlatbox
!cdo sellonlatbox,-11,-5,51,55.5 ../data_samples/netcdf/E-OBS/UK_monthly.nc tmp/Ireland_cdo.nc

cdi  warning (cdfScanVarAttr): NetCDF: Variable not found - time_bnds
cdo    sellonlatbox: Processed 5264064 values from 3 variables over 888 timesteps [0.18s 71MB]


In [30]:
# Task: Explore the content of your new files with ncdump and ncview
!ncdump -h tmp/Oxford_cdo.nc
!ncview tmp/Oxford_cdo.nc

netcdf Oxford_cdo {
dimensions:
	time = UNLIMITED ; // (888 currently)
	lon = 1 ;
	lat = 1 ;
variables:
	int time(time) ;
		time:standard_name = "time" ;
		time:long_name = "Time in days" ;
		time:units = "days since 1950-01-01 00:00" ;
		time:calendar = "standard" ;
		time:axis = "T" ;
	double lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
	double lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
	float pp(time, lat, lon) ;
		pp:standard_name = "air_pressure_at_sea_level" ;
		pp:long_name = "sea level pressure" ;
		pp:units = "hPa" ;
		pp:_FillValue = -9999.f ;
		pp:missing_value = -9999.f ;
		pp:cell_methods = "time: mean" ;
	float rr(time, lat, lon) ;
		rr:standard_name = "thickness_of_rainfall_amount" ;
		rr:long_name = "rainfall" ;
		rr:units = "mm" ;
		rr:_FillValue = -9999.f ;
		rr:missing_value = -9999.f ;
		rr:cell_meth

**Question: Can you see an increase in temperature over the period in Oxford?**

### Statistical operations

In [36]:
# Compute the yearly averaged time series using cdo yearmean
!cdo yearmean tmp/Oxford_cdo.nc tmp/Oxford_yearly_cdo.nc

cdo    yearmean:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 91cdo    yearmean: Processed 2664 values from 3 variables over 888 timesteps [0.10s 35MB]


In [37]:
# Visualize the new file with ncview
!ncview tmp/Oxford_yearly_cdo.nc

Ncview 2.1.8 David W. Pierce  8 March 2017
http://meteora.ucsd.edu:80/~pierce/ncview_home_page.html
Copyright (C) 1993 through 2015, David W. Pierce
Ncview comes with ABSOLUTELY NO WARRANTY; for details type `ncview -w'.
This is free software licensed under the Gnu General Public License version 3; type `ncview -c' for redistribution details.

Note: no Ncview app-defaults file found, using internal defaults
^C


**Question: Can you see an increase in yearly temperature over the 1950-2023 period in Oxford?**

In [35]:
# Compute the average (over time) temperature in Ireland
!cdo timmean tmp/Ireland_cdo.nc tmp/Ireland_avg_cdo.nc

cdo    timmean:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 910cdo    timmean: Processed 1150848 values from 3 variables over 888 timesteps [0.11s 46MB]


In [34]:
# Task: Explore the content of your new file with ncdump and ncview. Check that you get the same values as with nco before.
!ncdump -h tmp/Ireland_avg_cdo.nc
!ncview tmp/Ireland_avg_cdo.nc

netcdf Ireland_avg_cdo {
dimensions:
	time = UNLIMITED ; // (1 currently)
	bnds = 2 ;
	longitude = 24 ;
	latitude = 18 ;
variables:
	int time(time) ;
		time:standard_name = "time" ;
		time:long_name = "Time in days" ;
		time:bounds = "time_bnds" ;
		time:units = "days since 1950-01-01 00:00" ;
		time:calendar = "standard" ;
		time:axis = "T" ;
	double time_bnds(time, bnds) ;
	double longitude(longitude) ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "Longitude values" ;
		longitude:units = "degrees_east" ;
		longitude:axis = "X" ;
	double latitude(latitude) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "Latitude values" ;
		latitude:units = "degrees_north" ;
		latitude:axis = "Y" ;
	float pp(time, latitude, longitude) ;
		pp:standard_name = "air_pressure_at_sea_level" ;
		pp:long_name = "sea level pressure" ;
		pp:units = "hPa" ;
		pp:_FillValue = -9999.f ;
		pp:missing_value = -9999.f ;
		pp:cell_methods = "time: mean" ;
	float rr(time, latitude

In [17]:
# Task: Explore the content of your new file with ncdump and ncview
# Remark the time dimension now shows 74 time steps for the 74 years of the dataset.
!ncdump -h tmp/T_France_fldmean_yearmean.nc
!ncview tmp/T_France_fldmean_yearmean.nc

ncdump: tmp/T_France_fldmean_yearmean.nc: No such file or directory
Ncview 2.1.8 David W. Pierce  8 March 2017
http://meteora.ucsd.edu:80/~pierce/ncview_home_page.html
Copyright (C) 1993 through 2015, David W. Pierce
Ncview comes with ABSOLUTELY NO WARRANTY; for details type `ncview -w'.
This is free software licensed under the Gnu General Public License version 3; type `ncview -c' for redistribution details.

ncview: can't open file tmp/T_France_fldmean_yearmean.nc : No such file or directory


### Chaining commands

`cdo` commands can be chained, but make sure to use the dashes, and be mindful of the order of processes.

In [18]:
# Task: Run the previous workflow chaining all the commands
!cdo -yearmean -fldmean -sellonlatbox,-4.7,7.8,42.5,51.0 ../data_samples/netcdf/E-OBS/tg_ens_mean_0.25deg_reg_v29.0e.nc tmp/chain.nc

cdo(1) fldmean: Process started
cdo(2) sellonlatbox: Process started
HDF5-DIAG: Error detected in HDF5 (1.14.3) thread 1:
  #000: H5F.c line 660 in H5Fcreate(): unable to synchronously create file
    major: File accessibility
    minor: Unable to create file
  #001: H5F.c line 614 in H5F__create_api_common(): unable to create file
    major: File accessibility
    minor: Unable to open file
  #002: H5VLcallback.c line 3605 in H5VL_file_create(): file create failed
    major: Virtual Object Layer
    minor: Unable to create file
  #003: H5VLcallback.c line 3571 in H5VL__file_create(): file create failed
    major: Virtual Object Layer
    minor: Unable to create file
  #004: H5VLnative_file.c line 94 in H5VL__native_file_create(): unable to create file
    major: File accessibility
    minor: Unable to open file
  #005: H5Fint.c line 1852 in H5F_open(): unable to open file: name = 'tmp/chain.nc', tent_flags = 13
    major: File accessibility
    minor: Unable to open file
  #006: H5FD.

In [19]:
# Task: Explore the content of your new file with ncdump and ncview
!ncdump -h tmp/chain.nc
!ncview tmp/chain.nc

ncdump: tmp/chain.nc: No such file or directory
Ncview 2.1.8 David W. Pierce  8 March 2017
http://meteora.ucsd.edu:80/~pierce/ncview_home_page.html
Copyright (C) 1993 through 2015, David W. Pierce
Ncview comes with ABSOLUTELY NO WARRANTY; for details type `ncview -w'.
This is free software licensed under the Gnu General Public License version 3; type `ncview -c' for redistribution details.

ncview: can't open file tmp/chain.nc : No such file or directory


In [20]:
# Task: Compare the file built step by step and the one with command chaining using cdo diff
# Remark some records might differ but the small values indicate averaging errors more than a real difference in the outcomes.
!cdo diff tmp/T_France_fldmean_yearmean.nc tmp/chain.nc

cdo    diff: Open failed on >tmp/T_France_fldmean_yearmean.nc<
             No such file or directory


### Wrap-up on terminal use
There are two reasons for using the terminal to explore and manipulate your files before going to Python: 
1. To get a quick look at a file and checking that it contains what you want before you open Python
2. Data manipulation with `nco` and `cdo` (see below) are much more efficient than in Python. For heavy file, it is recommended to first reduce data dimensionality and weight with command-line tools before you open the files in Python. 

In [21]:
# Task: Compare the weight of the full temperature file versus the one where you selected only one country
# Remark: Pre-processing the file (in that case extracting the region of interest) can greatly reduce file size, 
# making it faster to load and manipulate in Python.
!ls -lhS ../data_samples/netcdf/E-OBS/tg_ens_mean_0.25deg_reg_v29.0e.nc
!ls -lhS tmp/T_France.nc

-rw-r--r--  1 bourdin  staff    61M  6 Aug 15:35 ../data_samples/netcdf/E-OBS/tg_ens_mean_0.25deg_reg_v29.0e.nc
ls: tmp/T_France.nc: No such file or directory


In [22]:
# Task: Remove the files created in the tmp folder
!rm -f tmp/*

zsh:1: no matches found: tmp/*


As you can see, some tools are redundant, and it is up to you to decide which tool works best for you. `nco` and `cdo` contain many more functions, and it is very likely you will be able to do most of any pre-processing with these tools. Do not forget to always check step by step what each function is doing.